# Clickbait Detection: Logistic Regression (Scaled) & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook αναπτύσσουμε και αξιολογούμε ένα μοντέλο **Logistic Regression**.

### Γιατί χρειάζεται Scaling;
Σε αντίθεση με τα δενδρικά μοντέλα (π.χ. Gradient Boosting) που είναι αδιάφορα στην κλίμακα, η Logistic Regression επηρεάζεται έντονα από το εύρος των τιμών, καθώς χρησιμοποιεί αλγορίθμους βελτιστοποίησης (solvers όπως `saga`, `lbfgs`) που βασίζονται σε παραγώγους.

Για να συγκλίνει το μοντέλο και να αποδώσει σωστά στα UMAP embeddings, εφαρμόζουμε **`StandardScaler`** (z-score normalization).

## Στόχοι
1. **Φόρτωση Δεδομένων**: Χρήση των UMAP-500 parquets.
2. **Βελτιστοποίηση (Optuna)**: Εύρεση παραμέτρων (`C`, `penalty`, `solver`) με ενσωματωμένο Scaling ανά trial.
3. **Champion Model**: Εκπαίδευση του τελικού μοντέλου στο σύνολο Train+Valid.
4. **Αποθήκευση Artifacts**: Αποθήκευση του Μοντέλου ΚΑΙ του Scaler (απαραίτητο για χρήση σε νέα δεδομένα).

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import pickle
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Απόκρυψη FutureWarnings για καθαρό output
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# Ρύθμιση Paths για να βρούμε το mlflow_helper και τα δεδομένα
# Υποθέτουμε ότι το notebook είναι στο φάκελο: src/Notebooks
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## Φόρτωση Δεδομένων

Φορτώνουμε τα Train/Valid/Test sets.
**Σημείωση:** Εδώ φορτώνουμε τα "ωμά" δεδομένα. Το Scaling θα εφαρμοστεί δυναμικά στα επόμενα βήματα για να αποφύγουμε το Data Leakage (διαρροή πληροφορίας από το test set).

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"⏳ Έναρξη φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features
        feature_cols = [c for c in df.columns if c.startswith("umap_")]
        if not feature_cols:
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 2. Εντοπισμός Label
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        if label_col is None:
             raise ValueError(f"Δεν βρέθηκε label στήλη στο {filename}")

        # Διαχωρισμός X, y
        if label_col in feature_cols:
            feature_cols.remove(label_col)

        y = df[label_col].values.astype(int)
        X = df[feature_cols].values.astype(np.float32)

        loaded_data[name] = (X, y)
        print(f"   ✅ [{name}] Loaded: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## Βελτιστοποίηση Υπερπαραμέτρων (Tuning)

Ορίζουμε τη συνάρτηση `objective` για το Optuna.
**Κρίσιμο Βήμα:** Εδώ δημιουργούμε τον `StandardScaler` και τον κάνουμε `.fit()` **μόνο** στο Training κομμάτι του εκάστοτε trial. Το Validation set γίνεται απλώς `.transform()`.

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # --- 1. Hyperparameters Space ---
    C = trial.suggest_float("C", 1e-4, 1e2, log=True)
    penalty = trial.suggest_categorical("penalty", ["l2", "l1", "elasticnet"])
    solver = trial.suggest_categorical("solver", ["lbfgs", "liblinear", "saga"])
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])
    max_iter = trial.suggest_int("max_iter", 500, 3000) # Αυξημένο για σύγκλιση

    # --- 2. Constraints (Pruning invalid combinations) ---
    if penalty == "elasticnet" and solver != "saga":
        raise optuna.exceptions.TrialPruned()
    if penalty == "l1" and solver == "lbfgs":
        raise optuna.exceptions.TrialPruned()

    l1_ratio = None
    if penalty == "elasticnet":
        l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)

    # --- 3. Scaling (Εντός του Trial) ---
    scaler = StandardScaler()
    X_tr_sc = scaler.fit_transform(X_tr) # Fit στο Train
    X_v_sc = scaler.transform(X_v)       # Transform στο Valid

    # --- 4. Model Setup ---
    params = dict(
        C=C, penalty=penalty, solver=solver,
        class_weight=class_weight,
        max_iter=max_iter, random_state=42
    )
    if l1_ratio is not None:
        params["l1_ratio"] = l1_ratio

    model = LogisticRegression(**params)

    # --- 5. Training & Eval ---
    t0 = time.time()
    model.fit(X_tr_sc, y_tr)
    train_time = time.time() - t0

    preds = model.predict(X_v_sc)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    metrics = {"val_f1": f1, "val_accuracy": acc, "training_time_sec": train_time}

    # Καταγραφή στο MLflow
    params_to_log = params.copy()
    params_to_log["scaler"] = "StandardScaler" # Σημείωση ότι χρησιμοποιήθηκε scaler

    mlflow_helper.log_optuna_trial(trial, params_to_log, metrics, model, run_name_prefix="LR_Trial")

    return f1

## Εκτέλεση Πειράματος & Champion Model

1. Τρέχουμε το **Tuning** (20 trials).
2. Επιλέγουμε τις βέλτιστες παραμέτρους.
3. **Επανεκπαιδεύουμε** στο πλήρες σετ (Train + Valid).
4. **Σημαντικό:** Αποθηκεύουμε τον `scaler.pkl` μαζί με το μοντέλο, ώστε να μπορεί να χρησιμοποιηθεί αργότερα (π.χ. στο evaluation script).

In [ ]:
EXPERIMENT_NAME = "Clickbait_LogReg_UMAP500_Optuna_Scaled"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\n🚀 Έναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Tuning ---
print("\n🔍 [ΦΑΣΗ 1] Hyperparameter Tuning...")
with mlflow.start_run(run_name="LR_Hyperparameter_Tuning"):
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("scaler", "StandardScaler")

    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda t: objective(t, X_train, y_train, X_val, y_val),
        n_trials=20,
        catch=(optuna.exceptions.TrialPruned,)
    )

    print("🏆 Best F1:", study.best_value)
    print("🏆 Best Params:", study.best_params)


# --- ΦΑΣΗ 2: Champion Model ---
print("\n👑 [ΦΑΣΗ 2] Champion Model Training...")
with mlflow.start_run(run_name="LR_Champion_Model") as final_run:
    best = study.best_params

    # 1. Προετοιμασία Δεδομένων (Full Train)
    X_full = np.vstack([X_train, X_val])
    y_full = np.concatenate([y_train, y_val])

    # 2. Final Scaling (Fit στο Full Dataset)
    scaler = StandardScaler()
    X_full_sc = scaler.fit_transform(X_full)
    X_test_sc = scaler.transform(X_test) # Transform το Test με τον ίδιο scaler

    # 3. Setup Model
    champion_params = dict(
        C=best["C"], penalty=best["penalty"], solver=best["solver"],
        class_weight=best["class_weight"],
        max_iter=best["max_iter"], random_state=42
    )
    if best["penalty"] == "elasticnet":
        champion_params["l1_ratio"] = best["l1_ratio"]

    final_model = LogisticRegression(**champion_params)

    # 4. Training
    t0 = time.time()
    final_model.fit(X_full_sc, y_full)
    final_train_time = time.time() - t0

    # 5. Logging
    mlflow.log_params(champion_params)
    mlflow.log_param("scaler", "StandardScaler")

    # Αποθήκευση του Scaler ως artifact!
    with open("scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)
    mlflow.log_artifact("scaler.pkl")

    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # 6. Evaluation
    print("📈 Αξιολόγηση στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test_sc, # Προσοχή: Δίνουμε τα SCALED δεδομένα
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\n✅ Ολοκληρώθηκε. Run ID: {final_run.info.run_id}")